In [1]:
import time
import signal
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import SuguruSolvers
import SASolver

In [2]:
SOLVERS = [SuguruSolvers.DeterministicEngine, SASolver.SASolver, SuguruSolvers.BacktrackSolver]

def timeout_handler(signum, frame):
    raise Exception('EndOfTime')

def get_instances():
    # Load instances
    instances = list()
    for root, dirs, files in os.walk('./instances'):
        for filename in files:
            file_path = os.path.join(root, filename)
            instances.append(file_path)
    return instances

def parse_suguru_binary(path):
    if not os.path.isfile(path):
        raise Exception('Error: invalid file path')

    # Load from file
    with open(path, 'rb') as fp:
        rows = int.from_bytes(fp.read(2))
        cols = int.from_bytes(fp.read(2))
        arr = np.fromfile(fp, dtype=np.int16).reshape(3, rows, cols)
    return arr

def evaluate_solved_count():
    global SOLVERS
    results = dict()
    for solver in SOLVERS:
        instances = get_instances()
        for instance in instances:
            grid, _, regions = parse_suguru_binary(instance)

            # Get puzzle statistics
            tips = np.count_nonzero(grid != 0)
            size = grid.size
            region_count = np.max(regions.flatten())
            
            # 5 Minute timeout
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(300)

            # Solving using the current solver
            solved = False
            timeout = True
            try:
                s = solver(grid, regions)
                start = time.perf_counter()
                solved = s.solve()
                end = time.perf_counter()
                elapsed = end - start
            except Exception as e:
                solved = False
                timeout = True
                
            if solver not in results:
                results[solver] = {'solved':list(), 'elapsed':list(), 'puzzle':list(), 'tips':list(), 'size':list(), 'region_count':list(), 'timeout':False}
            
            if solved:
                results[solver]['solved'].append(1)
                results[solver]['elapsed'].append(elapsed)
            else:
                results[solver]['solved'].append(0)
                results[solver]['elapsed'].append(np.inf)

            results[solver]['tips'].append(tips)
            results[solver]['puzzle'].append(instance)
            results[solver]['size'].append(size)
            results[solver]['region_count'].append(region_count)
            results[solver]['timeout'] = timeout
            
            if len(results[solver]['solved']) % 100 == 0:
                print('[i] Tested: ', len(results[solver]['solved']))
            
    return results


In [ ]:
r = evaluate_solved_count()

[i] Tested:  100
[i] Tested:  200
[i] Tested:  300
[i] Tested:  400
[i] Tested:  500
[i] Tested:  600
[i] Tested:  700
[i] Tested:  800
[i] Tested:  900
[i] Tested:  1000
[i] Tested:  1100
[i] Tested:  1200
[i] Tested:  1300
[i] Tested:  1400
[i] Tested:  1500
[i] Tested:  1600
